![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/open-source-nlp/14.0.GPT2_Transformer_In_SparkNLP.ipynb)

# GPT2Transformer: OpenAI Text-To-Text Transformer

![image.png](https://miro.medium.com/max/1400/1*UZTblFMn8oWfmqBMQhUOdA.jpeg)

[GPT-2](https://openai.com/blog/better-language-models/)  displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where the model is primed with an input and it generates a lengthy continuation.

Pretrained models can be loaded with `pretrained()` of the companion object:

**GPU MODE** is highly reccomended, since this is a very large model.

## Colab Setup

In [ ]:
! pip install -q pyspark==3.4.1 spark-nlp==5.3.2

In [2]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

**Make sure to Enable GPU Mode and High RAM**

In [3]:
# Comment out this line  and uncomment the next one to enable GPU mode and High RAM

spark = sparknlp.start()

# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 5.3.2
Apache Spark version: 3.4.1


**GPT2 Models In Spark NLP**

*   `gpt2`
*   `gpt2_medium`
*   `gpt2_distilled`
*   `gpt2_large`

The default model is `"gpt2"`, if no name is provided. For available pretrained models please see the [Spark NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=gpt2)



## GPT2 Pipeline

Now, let's create a Spark NLP Pipeline with `gpt2_medium` model and check the results.

In [4]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

gpt2 = GPT2Transformer.pretrained("gpt2_medium") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setMinOutputLength(25) \
    .setOutputCol("generation")

data = spark.createDataFrame([["My name is Leonardo."]]).toDF("text")

pipeline = Pipeline().setStages([documentAssembler,gpt2]).fit(data)

result = pipeline.transform(data)
result.select("generation.result").show(truncate=False)

gpt2_medium download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ My name is Leonardo. I'm a student at the University of California, Berkeley. I've been studying computer science for the past two years. I have a PhD in computer scienc

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [ ]:
sample_texts= [[1, "Mey name is  Leonardo"],
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"],
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [ ]:
gpt2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                                            |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Mey name is  Leonardo Da Vinci. The first Da Vinzci painting depicts a young person holding his hands over their heads in meditation; the second painting depicts the man in meditation sitting down a

### Changing the Transformer's task

Now, we change the task of Transformer. We can verify some informations to GPT-2 by setting `.setTask()` parameter as **"Is it true that"**. <br/>
We give a text to the model by setting `setTask("Is it true that")` and model adds the "Is it true that" expression at the beginning of the sentence and generates sentences.

In [ ]:
gpt2.setTask("Is it true that").setMaxOutputLength(50)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
sample_text= [[1, "Donald Trump is rich?"],
              [2, "Pink Floyd is rock band?"]]

sample_df= spark.createDataFrame(sample_text).toDF("id", "text")

result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                             |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it true that Donald Trump is rich? And the rich aren't wealthy, according to the data.\n\nTrump's wealth is the richest among the presidential candidates.\n\n\n\nHere is a map of who is making more in terms of income than]|
|2  |[ Is it true that Pink Floyd is rock band?\n\nThe best rock

# Some Helper Methods for generating and printing generation results

In [5]:
from typing import List
def prompts_to_spark_df(prompts,spark=spark):
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")

def generate_with_gpt2(gpt_pipe, prompts,task='generate', print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else :
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = pipeline.transform(df)
  df = df.select("id",'text', "generation.result").toPandas()

  if print:
    print_generation_results(df)
  return df

def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore GPT Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words.
The probability distribution is conditioned on all previous tokens in a text to generate the next token.

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions :

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters :

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable.


These parameters are shared by all method :
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf>
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: True)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for 

### Play with temperature
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `gpt2.setDoSample(True)` to have non-deterministic results

In [ ]:
text = """Hello my name is GPT2, I love to """
data = [text, text,text,text,text ]
gpt2.setMaxOutputLength(200)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ive made a new thread and asked some questions in the past few minutes or something like that? I can't find it on the forums yet.


Also is it true Hello my Name is, I want to make a thread asking questions, like what kind of games you have, if it have been out for a while, if you still play the game or not, or if you just love it and play a few times and don't think you are in the top 2 of the leaderboards. I just finished up my first game on Steam. Also I want help on how to play it, and if you can help please, please please do so please please please post the info here on here. Thanks!!


Thank you all for your patience!!

Last edited by Miho on Apr 20, 2013, 4:40:55 PM Posted by Miboo

on Alpha Member on Quote


Example 1: _

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.75)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to _______, I will be able to do _______.

I've seen this in the comment section when someone posted a reply, but it is something I've never heard before.
.


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ____.

[12:07:45] Name: Zippo Green (The Answer) XP: 500 XP: 1 Gold: 0 Gold: 1 XP: 125 Item Hash: 098A6F6AA

Name: Power Nap Chute XP: 50 XP: 20 Armor Class Resistances Base 20 Fire 0% Cold 0% Crushing 0 Missile 0 Electricity 0% Acid 0% Piercing 0 Slashing 0 Magic 50% Magical fire 0% Magical cold 0% THAC0: 0 Slapping 0

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.5)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to  play the game, but I am not able to play on the main server. I have been playing on the server for about a month, and I am experiencing the same issues as the other players. I know that this is not the case for everyone, but for me it is. I am wondering if there is a way to fix this, or if it is something that I can try to fix. I would really appreciate any help in this regard. Thanks in advance.
Hello my name IS GPT1, I am playing on my main server and I have the same issue. I hope that this helps.
Thank you for your concern. I do not know if this is related to the issue you are having.
I am playing the game on the game server and the same problem has also been happening on the same server.
Hi, I have a question about this. I 

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.25)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline,data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

I'm a newbie to the world of Linux. I've been using Linux for about a year now, and I'm still learning. I'm looking for a good, stable Linux distribution that I can use for my daily work. I want to be able see my desktop in a way that I never could before. I also want to have a nice, clean, and easy-to build Linux system.
 -GPT2
 a guest Sep 15l, 2014 1,836 Never 2/15 ea
 'I'm looking to buy a new laptop and I need a good Linux distro. I have been using Ubuntu for a while now, but I'm not sure if it's the best distro for me. I would like to know if there are any distils that I should try out. I am a new user and I don''t know what to expect. I


Example 1: ____________________________________________________________________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! But is it true? Can I install some packages of Debian 8 by a community software to work my Ubuntu/Mac systems from here? It is, this site also offer, in general, several of its projects under license GNU GPLv3. Are you in trouble as GPT3 version and all software with similar or older copyright will be rejected at the last because of different code? So in addition to install Linux software of its Debian equivalent as I need only do something more serious I recommend Linux Debian to do my need because we could build new life without software restrictions by use of some third party toolchain of free/open source Software . The question on here might be very tricky when my main motivation from here is to keep with our freedom wit

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.0001)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ________________, HiGTPiRzCq4Nc7H3R7c1jdPtPX9P5p4x
It does however work just okay so its definitely more like how an eee can feel with gppa . Not great yet for someone to install from usb though if only one guy or so gets hold in him!So that might ormay explain, just so no surprises:My eemup does appear better however...it appears slightly weaker dueto what a bunch nuke said that we already already found here as stated a previous week..If they will keep talking then more testing are surely being ordered soon enoughI've made more sure, thanks very kindly people are interested :),Hope everything go very right this new development cycle in linux . This project also deserves our applause if i'll continue its working !It just got ver

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


# GPT2 Use Cases

These are some examples to give you an idea of what GPT2 is capable of

## GPT for Programming Code

### GPT and Scala

In [ ]:
# Hello world in Scala
scala_code = """
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, scala_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
I could use the same approach to generate a message to output from the function that creates a HelloWorld world and send it to your mail program via standard send command. For example, you can write the following code:
class HelloWorld{
public:
bool operator==(const HelloWorld& a, const HelloWorld & b) const;
bool equal(const std::string& s1,const std ::string& t1,bool b){
return (a == b) && (s1[0] == t1[1],s1 [2] == s1 [3]);
} // Output: Hello, world
};
To generate the message:
main()
Now if we have compiled the hello world program and run it, we'll see the following:


[TestMethod@2f3c9e9b0d]

public override void TestTest(Object obj, string message){

StringBuilder 

,id,text,result
0,0,\nobject Hello {\n def main(args: Array[Str...,[ Is it true that \nobject Hello {\n def ma...


### GPT and Python

In [ ]:
# adding 3 numbers in python
py_code = """
def my_func(a,b,c):
  return a+b+c
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, py_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def my_func(a,b,c):
  return a+b+c

"""

def __str__ ( self ):

return _runt( self )

# The function arguments should be passed in the order that they are seen in the Python string,

// and this must be done from the top:

if self ._strtolist:
.compare( self .__str__()[: 2 ])

else :

.lower();

__str_cmp ( self . __str_get( self ._args), self . _compare())

self ._str_tolists = None

try :
.append( None )
 "

"

except ValueError as e:
,e

raise

@ property

class _run_comparer ( runt ):
, runt

The _runsoc function allows us to convert values into a more useful form. This

function converts Python values in two ways.

First, the values can be converted to a string and then the converted string can be concatenated

to give a string value. For example, in the 

,id,text,result
0,0,"\ndef my_func(a,b,c):\n return a+b+c\n","[ Is it true that \ndef my_func(a,b,c):\n ret..."


In [ ]:
# breadth-first search
bfs_python_code = """
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
"""


gpt2.setMaxOutputLength(1000)
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, bfs_python_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
if bfs: 
queue.push(node.data)
else:
queue.push([])
print(queue.pop()[0:])

 If you've followed all of this, now it's time to test a different version of this program. I will start with an empty set, which I call "Empty". If I had one more node (which I will call 1) in the set with a graph of its own (which is called an  "empty graph"), I would call that  and call it empty as well, just so I could see what happens. In the real world, we'll have to write the code that takes the first node  in the

,id,text,result
0,0,"\ndef bfs(visited, graph, node):\n visited.ap...","[ Is it true that \ndef bfs(visited, graph, no..."


## Generate Product Recommendations

We can start a base list of top 10 anything and GPT2 will complete it for us!      
This is very useful to reccomendation and search engines.

### Top 10 Movie List

In [ ]:
movie_list = """My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4."""

gpt2.setTemperature(1)
gpt2.setMaxOutputLength(200)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. Independence Day
5. Terminator Genisys (2008)
6. Avatar: The Last Airbender
7. Transformers
8. Titanic
9. Avatar 2: The Secret of Avatar
10. A Clockwork Orange




,id,text,result
0,0,My top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My top 10 movie list :\n1. T...


How **not** to generate text with GPT2

Since every new word is conditioned on every previous **character**, things like **white spaces** and **new lines** can skew the sampling distribution into an unintended direction

In [ ]:
# NOTE! We have malformed the input text and added a newline.
# This will Confuse GPT2, it might think the list is over and probability distribution changes accordingly
movie_list = """My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.

"""
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.


1-10 :
7. Blade Runner 1
8. The Dark Knight


9-12 :

1: The Shining (1980) 2: Casablanca (1980)-3: The Princess Bride (1981) 4: The Godfather 3: Casascius (1982)


13- 15 :
.

4: The Matrix Revolutions


16- 20 :
 and

5: Blade Runner: The Complete Series 4: Terminator 2: Judgment Day


The following films are included in this list as well :





,id,text,result
0,0,My Top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My Top 10 movie list :\n1. T...


In [ ]:
movie_list = """My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4.  Super Mario Bros.: Mario Bros., Super Smash Bros.
5. Transformers: Age of Extinction
6. The Dark Knight Rises
7. The Blair Witch Project
8. The Bourne Ultimatum
9. The Avengers
10. The Big Sick
In conclusion, if you have a list of worst movies ever, please share it with us. We'd love to know what movies you have to say about it!


Have something bad to say to me? Tweet me a list and we'll do our best to keep it classy and relevant.




,id,text,result
0,0,My Top 10 worst movie list :\n1. The Matrix 4\...,[ Is it true that My Top 10 worst movie list :...


### Top 10 Game List

In [ ]:
game_list = """My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4.  StarCraft II: Wings of Liberty
5. Doom
6. Super Mario 64
7. Final Fantasy IV
8. Metroid Prime
9. Final Fantasy V
10. Wario's Mansion
And so on! I have spent thousands of hours watching and reading about video games, reading articles about them on various video game blogs, and reading books about them. But when it came time to select the Top 10 Video Games of All Time - and I am happy to say that it was actually fun! - I knew I had to get the very first video game I own and I had no idea what I would get myself into. So I went to the gaming section at my local Walmart. I was going to buy two games, just to be prepared to buy the first game. The price




,id,text,result
0,0,My Top 10 video games of all time list :\n1. H...,[ Is it true that My Top 10 video games of all...


### Top 10 Book List

In [ ]:
game_list = """My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4.  The Secret Lives of Famous People
5. The Seven Year Itch
6. A History of Violence
7. The Secret War of the United States
8. The Rise of the Warrior Cop
9. A Guide to the Dark Side of the Moon
10. Top Ten Books of All Time




,id,text,result
0,0,My Top 10 books of all time list :\n1. 1964\n2...,[ Is it true that My Top 10 books of all time ...


## SEO Text / Content Writing / Copy Writing
Lets generate some texts for our new imaginary shopify store!
We will sell some hoodies, soap and beard products.

Based on some base templates, we can generate high quality marketing/seo texts

In [ ]:
# from https://www.shopify.com/blog/8211159-9-simple-ways-to-write-product-descriptions-that-sell

# Lets define some base texts, from which we will generate new texts

hoodie = """Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! """

soap  = """Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent."""

beard = """ Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, """
gpt2.setTemperature(1)
gpt2.setDoSample(True)

GPT2TRANSFORMER_729a8e69e804

#### Marketing Texts for Hoodies

In [ ]:
# Lets generate some marketing texts for our hoodies
generate_with_gpt2(pipeline, [hoodie,hoodie,hoodie,hoodie,hoodie,hoodie] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! ____________ You can now purchase in your own store or purchase online from Amazon.com


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! Â

Can*t Stop Buying Plants?!

"It is my love and loyalty to my plant that keeps me alive as well as alive, but my passion is my plant love. I want to be an oodie to my tree and be a 

,id,text,result
0,0,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
1,1,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
2,2,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
3,3,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
4,4,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
5,5,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...


#### Marketing Texts for Soap

In [ ]:
# Lets generate some marketing texts for our soap
generate_with_gpt2(pipeline, [soap,soap,soap,soap,soap,soap] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent.
This super-damp, waterproof bar features 100% Natural and Organic Beeswax that's non-greasy, non-toxic, and anti-perspirational. The patented, patented waterproof technology makes your shower showering in the morning feel like it just stepped off a plane (and back again). The non-comedogenic, ultra-lightweight formula creates a rich, luxurious shower that's a breath of fresh air.
Pine sap is known to be antibacterial and antiinflammatory, while it's proven to reduce the appearance of pores around eyes, mouth, and nose. A combination of 100% natural extract and the

,id,text,result
0,0,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
1,1,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
2,2,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
3,3,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
4,4,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
5,5,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."


#### Marketing Texts for beard products

In [ ]:
# Lets generate some marketing texts for our beard products
generate_with_gpt2(pipeline, [beard,beard,beard,beard,beard,beard] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that  Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control,  even without the need for a bulky brush.  It also features an ultra-soft formula with a super-soft feel, to help you stay calm and relaxed when styling.   For the most natural and natural looking finish, just use our  Beardbrand  Styling Gel - a natural looking, gel formula with high hydration, hydration levels for more hydration. 
Beardbrand Styler: a natural hair care product that looks natural without compromise


Example 1: ________________________________________________________________________________________________________________________________________________________

,id,text,result
0,0,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
1,1,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
2,2,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
3,3,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
4,4,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
5,5,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."


## Song Lyrics

#### Last Christmas

In [ ]:
last_christmas = """
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, last_christmas,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
I've had so much to offer you
Christmas was always the best
The Christmas tree I've got now,
The presents I gave to you, I'd give to you any day
But we'll never have this Christmas again
Christmas is my only wish and it's always been
I'm here for you, for you alone
Christmas tree you've given me, I'm here by your side,
It was always Christmas tree before, we always celebrated Christmas together
You know it was all so special
You're all a part of Christmas, it's a special thing
And it's Christmas again, and I'm gonna give you another one
But this one you'll never hear the end of
It's Christmas time, and Christmas time is m

,id,text,result
0,0,"\nLast Christmas, I gave you my heart\nBut the...","[ Is it true that \nLast Christmas, I gave you..."


#### Fresh prince of GPT

In [ ]:
bel_air = """
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, bel_air,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
Where a lot of people have been
A lot of them have been like-
A guy in his late 20s or late 40s
He got his dream job at Walmart-
So he started off at Walmart
But he wasn't happy, He had so much to prove
And he was really lonely
And at Walmart he'd spend his days
He'd spend nights playing around on the computer
And when his boss saw his face
He started yelling at him and called him a loser
When his boss didn't like him no less-
"What're you doing here?"
And so the boss came over to him
And they started arguing
And it turned from that to-
And we started fighting.
But I wanted m

,id,text,result
0,0,"\nNow, this is a story all about how\nMy life ...","[ Is it true that \nNow, this is a story all a..."


#### GPT Jackson

In [ ]:
thriller_lyrics = """
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
"""


gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, thriller_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
You're gonna come down on them
You need to learn the hard way




,id,text,result
0,0,"\nThey told him, ""Don't you ever come around h...","[ Is it true that \nThey told him, ""Don't you ..."


#### Real Slim GPT

In [ ]:
real_slim_shady = """
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, real_slim_shady,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
I'm not even the last one in Hollywood, I'm just the sixth
I've heard all of my friends say to die in your arms
My friends say that, but they're just gonna 

,id,text,result
0,0,"\nMay I have your attention, please?\nMay I ha...","[ Is it true that \nMay I have your attention,..."


#### GPT Rap God Bot

In [ ]:
rap_god_lyrics = """
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, rap_god_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot  
Rap-bot, Rap-man, Rap, Rapbot,Rap-dub,Rapbot,rap-bot
Rap bot, rap, rapbot, rap-dubs
If you have to say tha

,id,text,result
0,0,"\nLook, I was gonna go easy on you not to hurt...","[ Is it true that \nLook, I was gonna go easy ..."


## Fiction Writing

#### Fantasy Stories

In [6]:
unicorn_story = """In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, unicorn_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
The researchers also discovered that the animals were able to communicate with each other, and even to communicate in the form of music.
"The animals were very intelligent and they were able, through music, to communicate," said Dr. David M. Smith, a professor of zoology at the University of California, Santa Cruz.
Smith said that the findings are important because they show that the animal kingdom is not just a collection of animals, but also a collection that is evolving.
He said that it is important to understand that the study of animals is not l

,id,text,result
0,0,"In a shocking finding, scientist discovered a ...","[ In a shocking finding, scientist discovered ..."


In [ ]:
silly_story = """
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, silly_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!
This is a rare find and the people of the region are extremely proud of it!
There's more than just the  pile of blue peanut butter on a hilltop,
there's an entire cave, a world of mysteries waiting to become revealed!
Thanks to the awesome people who posted the photos of the 
hills and the cave, we now have a more complete picture of the place that

,id,text,result
0,0,\nBlue creatuers have been found in undergroun...,[ Is it true that \nBlue creatuers have been f...


#### Lovecraftian Horror Stories

In [ ]:
lovecraft_text = """
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, lovecraft_text,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall. It was a mass of black hair,

that fluttered up to the sky, with a wild, black face and a white throat,
, of white-grey eyes 

,id,text,result
0,0,\nAt this horror I sank nearly to the lichened...,[ Is it true that \nAt this horror I sank near...


In [ ]:
cthulu_intro = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, cthulu_intro,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little by little, and in many cases have had a far more harmful effect than our own ignorance can produce.
There is no great danger of the modern age making us too familiar with the past, and we must be careful lest this be the result, that it should lead us into a fatal trap.
As long as you remain ignorant, you will live in ignorance. But if once you become aware of what you are, you are free to act like a man.
I will give you a little more insight 

,id,text,result
0,0,"\nThe most merciful thing in the world, I thin...",[ Is it true that \nThe most merciful thing in...


#### Re-Write movie scripts

In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  Why?
, I created it. I was created.
- What? You weren't?
- No, because that's what you know, and that's my point.
The answer is that I created you. In order to create you I must create your mind. This machine I created with my mind has created us, and through us you are.
I am the matrix, you an

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\n\nNeo: Who are y...","[ Is it true that \nThe Architect: Hello, Neo...."


In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  Because it is necessary for your survival.  You are an Architect.
The first time you awaken, you are a human being. Â You wake up as a new form of life. You are aware of your body and the physical world around you. �You understand your body's natural laws and are aware enough to react with them 

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\nNeo: Who are you...","[ Is it true that \nThe Architect: Hello, Neo...."
